Model Store - will be creating:
* Model Group that will track all experiments and deployment history for our model, 
* Model Package that will record information on a specific model experiment and deployment
* Model Card that contains qualitative information for anyone who needs to maintain the model after its initial development.

This assignment should be relatively easy. Feel free to use lab code or code from your final project to complete this assignment. We simply want to take an existing model and add it to our Model Store.

In [55]:
import boto3
import sagemaker
import pandas as pd
from sagemaker.model_card import (
    ModelOverview, ObjectiveFunction, Function, ObjectiveFunctionEnum, FacetEnum,
    TrainingDetails, Metric, MetricTypeEnum, MetricGroup, EvaluationJob,
    IntendedUses, RiskRatingEnum, AdditionalInformation, ModelCard
)
# import numpy as np

In [7]:
# initialize 
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
boto_session = boto3.Session(region_name=region)

# sagemaker - understand the difference
sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)

#### Creating Model Group
Give your Model Group an informative name about what this group does, e.g. xgboost-breast-cancer-detection, 
and give it a brief but informative description of what this group does in a bit more detail (best practice is under ~250 chars).


In [ ]:
# Creating model package group
model_package_group_name = 'test-lab-model-group'
model_package_group_description = 'This group will be used to test out storage of models'
response = sagemaker_client.create_model_package_group(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageGroupDescription=model_package_group_description,
    Tags=[
        {
            'Key': 'MonthCreated',
            'Value': 'September'
        },
    ]
)

In [14]:
# Verify part 1
response = sagemaker_client.describe_model_package_group(
    ModelPackageGroupName=model_package_group_name
)
print(response)

{'ModelPackageGroupName': 'test-lab-model-group', 'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:936912055594:model-package-group/test-lab-model-group', 'ModelPackageGroupDescription': 'This group will be used to test out storage of models', 'CreationTime': datetime.datetime(2024, 9, 30, 22, 45, 2, 925000, tzinfo=tzlocal()), 'CreatedBy': {'IamIdentity': {'Arn': 'arn:aws:sts::936912055594:assumed-role/LabRole/SageMaker', 'PrincipalId': 'AROA5UJCPNUVM4RORLTYL:SageMaker'}}, 'ModelPackageGroupStatus': 'Completed', 'ResponseMetadata': {'RequestId': 'e3964823-70bd-4b1f-86d8-d759ed678992', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e3964823-70bd-4b1f-86d8-d759ed678992', 'content-type': 'application/x-amz-json-1.1', 'content-length': '455', 'date': 'Mon, 30 Sep 2024 22:46:42 GMT'}, 'RetryAttempts': 0}}


#### Creating Model Package

The Model Package will contain specific details about our current model. Our Model Package should document model deployment information (instance image, model data source i.e. our binary artifact, data source, any pre-processor or post-processor scripts, etc.). After we learn more about Model monitoring, we can also include model quality, model data quality, model bias and model explainability reports here too!

ref: https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-version.html
ref:

In [26]:
response = sagemaker_client.create_model_package(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageDescription='Initial model package deployed',
    InferenceSpecification={
        'Containers': [
            {
                'ContainerHostname': 'Container-1',
                'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1',
                'ModelDataUrl': 's3://sagemaker-us-east-1-936912055594/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-09-30-20-57-19/xgb-2024-09-30-20-57-19/output/model.tar.gz',
                'ProductId': '10',
                'Environment': {
                    'string': 'test'
                },
                'ModelInput': {
                    'DataInputConfig': 'batch'
                },
                'Framework': 'XGBOOST',
                'FrameworkVersion': '1.7',
                'NearestModelName': 'xgboost'
            },
        ],
        'SupportedTransformInstanceTypes': [
            'ml.m4.2xlarge'
        ],
        'SupportedRealtimeInferenceInstanceTypes': [
            'ml.m5.xlarge'
        ],
        'SupportedContentTypes': [
            'text/csv',
        ],
        'SupportedResponseMIMETypes': [
            'text/csv',
        ]
    }
)

In [34]:
sagemaker_client.list_model_packages(ModelPackageGroupName=model_package_group_name)


{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'test-lab-model-group',
   'ModelPackageVersion': 1,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:936912055594:model-package/test-lab-model-group/1',
   'ModelPackageDescription': 'Initial model package deployed',
   'CreationTime': datetime.datetime(2024, 9, 30, 23, 17, 52, 790000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed'}],
 'ResponseMetadata': {'RequestId': '37006c2a-3f00-4553-9186-2d58ce09768b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '37006c2a-3f00-4553-9186-2d58ce09768b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '323',
   'date': 'Mon, 30 Sep 2024 23:29:27 GMT'},
  'RetryAttempts': 0}}

In [36]:
# verify model package
response = sagemaker_client.describe_model_package(
    ModelPackageName='arn:aws:sagemaker:us-east-1:936912055594:model-package/test-lab-model-group/1'
)
print(response)

{'ModelPackageGroupName': 'test-lab-model-group', 'ModelPackageVersion': 1, 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:936912055594:model-package/test-lab-model-group/1', 'ModelPackageDescription': 'Initial model package deployed', 'CreationTime': datetime.datetime(2024, 9, 30, 23, 17, 52, 790000, tzinfo=tzlocal()), 'InferenceSpecification': {'Containers': [{'ContainerHostname': 'Container-1', 'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1', 'ImageDigest': 'sha256:f037aa7389a000dc611e723fb227d21a07cf495fd5e7bd8292a260ae101b5546', 'ModelDataUrl': 's3://sagemaker-us-east-1-936912055594/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-09-30-20-57-19/xgb-2024-09-30-20-57-19/output/model.tar.gz', 'Environment': {'string': 'test'}, 'ModelInput': {'DataInputConfig': 'batch'}, 'Framework': 'XGBOOST', 'FrameworkVersion': '1.7', 'NearestModelName': 'xgboost'}], 'SupportedTransformInstanceTypes': ['ml.m4.2xlarge'], 'SupportedRealtimeInferenceIns

#### Creating Model Card
The Model Card will contain qualitative details about our current model. The Model Card can contain a lot of information. At a minimum, it should contain details of what the model algorithm is, how the model was trained, what hyperparameters were used to train the model, what the input features for the model are, who the model owner is (you), what problem the model is trying to solve, intended uses of the model, evaluation details of the model, and so on.

ref: https://docs.aws.amazon.com/sagemaker/latest/dg/model-cards-create.html

In [53]:
model_overview = sagemaker.model_card.ModelOverview.from_model_name(
    model_name='sagemaker-xgboost-2024-09-30-21-06-06-556',
    sagemaker_session=sess,
)

objective_function = sagemaker.model_card.ObjectiveFunction(
    function=sagemaker.model_card.Function(
        function=sagemaker.model_card.ObjectiveFunctionEnum.MINIMIZE,
        facet=sagemaker.model_card.FacetEnum.LOSS,
    ),
    notes="An-explanation-about-objective-function",
)

print('model overview')
print(model_overview.model_id)
print(model_overview.inference_environment.container_image)
print(model_overview.model_artifact)

training_details = sagemaker.model_card.TrainingDetails.from_model_overview(
    model_overview=model_overview,
    sagemaker_session=sess,
    objective_function=objective_function,
    training_observations="Model-training-observations",
)
print('training details')
print(training_details.training_job_details.training_arn)
print(training_details.training_job_details.training_environment.container_image) 
print([{"name": i.name, "value": i.value} for i in training_details.training_job_details.training_metrics]) 

my_metric_group = sagemaker.model_card.MetricGroup(
    name="binary classification metrics",
    metric_data=[Metric(name="accuracy", type=sagemaker.model_card.MetricTypeEnum.NUMBER, value=0.5)]
)

evaluation_details = [
    sagemaker.model_card.EvaluationJob(
        name="Evaluation-job",
        evaluation_observation="Evaluation-observations",
        datasets=["s3://sagemaker-us-east-1-936912055594/DEMO-breast-cancer-prediction-xgboost-highlevel/validation"],
        metric_groups=[my_metric_group],
    )
]

print(evaluation_details)

intended_uses = IntendedUses(
    purpose_of_model="Purpose-of-the-model",
    intended_uses="The-intended-uses-of-this-model",
    factors_affecting_model_efficiency="Any-factors-effecting-model-efficacy",
    risk_rating=RiskRatingEnum.LOW,
    explanations_for_risk_rating="Explanation-for-low-risk-rating",
)

additional_information = AdditionalInformation(
    ethical_considerations="Any-ethical-considerations",
    caveats_and_recommendations="Any-caveats-and-recommendations",
    custom_details={"custom details1": "details-value"},
)

model overview
arn:aws:sagemaker:us-east-1:936912055594:model/sagemaker-xgboost-2024-09-30-21-06-06-556
['683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1']
['s3://sagemaker-us-east-1-936912055594/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-09-30-20-57-19/xgb-2024-09-30-20-57-19/output/model.tar.gz']
training details
arn:aws:sagemaker:us-east-1:936912055594:training-job/xgb-2024-09-30-20-57-19
['683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1']
[{'name': 'validation:logloss', 'value': 0.1469700038433075}, {'name': 'train:logloss', 'value': 0.0762299969792366}]


In [60]:
import json
model_card_name = 'MC-test-name'
card = ModelCard(
    name=model_card_name,
    status='Draft',  # Set the status of the model card
    model_overview=model_overview,
    training_details=training_details,
    intended_uses=intended_uses,
    evaluation_details=evaluation_details,
    additional_information=additional_information,
    sagemaker_session=sess,
)
card.create()

'arn:aws:sagemaker:us-east-1:936912055594:model-card/MC-test-name'

In [62]:
# Verify model card
response = sagemaker_client.describe_model_card(
    ModelCardName=model_card_name
)
print(response)

{'ModelCardArn': 'arn:aws:sagemaker:us-east-1:936912055594:model-card/MC-test-name', 'ModelCardName': 'MC-test-name', 'ModelCardVersion': 1, 'Content': '{"model_overview": {"model_id": "arn:aws:sagemaker:us-east-1:936912055594:model/sagemaker-xgboost-2024-09-30-21-06-06-556", "model_name": "sagemaker-xgboost-2024-09-30-21-06-06-556", "model_artifact": ["s3://sagemaker-us-east-1-936912055594/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-09-30-20-57-19/xgb-2024-09-30-20-57-19/output/model.tar.gz"], "inference_environment": {"container_image": ["683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1"]}}, "intended_uses": {"purpose_of_model": "Purpose-of-the-model", "intended_uses": "The-intended-uses-of-this-model", "factors_affecting_model_efficiency": "Any-factors-effecting-model-efficacy", "risk_rating": "Low", "explanations_for_risk_rating": "Explanation-for-low-risk-rating"}, "training_details": {"objective_function": {"function": {"function": "Minimiz